# VISUALISATION DES DONNÉES

In [2]:
# Importation des librairies nécessaires
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from  alimiji_filmns_sdk import MovieClient, MovieConfig
import time
import json
from collections import Counter, defaultdict
from pathlib import Path

/home/alimijileking/Documents/FILMNS_ANALYTICS/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [3]:
# Dossiers
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

In [5]:
# Connexion à l'API via le SDK

config = MovieConfig(movie_base_url="https://datascience-movie-backend.onrender.com")
client = MovieClient(config=config)

# Vérification que l'API est opérationnelle
client.health_check()

MOVIE_API_BASE_URL in MovieConfig init: https://datascience-movie-backend.onrender.com


{'message': 'API MovieLens opérationnelle'}

In [6]:
# Récupération des stats de l'API
analytics = client.get_analytics()
analytics

AnalyticsResponse(movie_count=9829, rating_count=100836, tag_count=3683, link_count=9742)

## Top 10 des genres par nombre de films

In [ ]:
"""
# Initialisation du compteur de genres
genre_counter = Counter()

# Paramètres pour le batching
limit = 1000
skip = 0

while True:
    batch = client.list_movies(skip=skip, limit=limit, output_format="dict")
    if not batch:
        break
    
    # On extrait les genres du lot et les compte
    for movie in batch:
        genres = movie.get("genres", "")
        genre_list = genres.split("|") if genres else []
        genre_counter.update(genre_list)
    
    skip += limit
    time.sleep(0.5)  # Pour respecter l’API

# Conversion du Counter en DataFrame
genre_df = pd.DataFrame(genre_counter.items(), columns=["genre", "count"])
genre_df = genre_df.sort_values("count", ascending=False).head(10)
display(genre_df)

# Bar chart horizontal
fig = px.bar(
    genre_df,
    x="count",
    y="genre",
    title="Top 10 genres par nombre de films",
    labels={"genre": "Genre", "count": "Nombre de films"},
    color="count",
    color_continuous_scale="viridis",
    orientation='h'  # ← clé pour l'affichage horizontal
)

fig.update_layout(
    yaxis={'categoryorder':'total ascending'},  # trie du haut vers le bas
    height=500
)

fig.show()

"""

,genre,count
6,Drama,4361
3,Comedy,3756
9,Thriller,1894
7,Action,1828
5,Romance,1596
0,Adventure,1263
8,Crime,1199
12,Sci-Fi,980
10,Horror,978
4,Fantasy,779


Nombre total de films par année (basée sur le titre)

In [17]:
####### Avec Système de mise en cache #########
api_movie_count = analytics.movie_count
print(api_movie_count)

genre_data_file = output_dir / "genre_df.parquet"
meta_file = output_dir / "meta.json"

# Lecture du fichier méta s'il existe
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cached_movie_count = meta.get("movie_count", 0)
else:
    meta = {}
    cached_movie_count = 0

# Décision : utiliser le cache ou recalculer
if genre_data_file.exists() and cached_movie_count == api_movie_count:
    print("Chargement des données depuis le cache...")
    genre_df = pd.read_parquet(genre_data_file, engine="fastparquet")

else:
    print("Mise à jour des données depuis l'API...")
    # Initialisation du compteur de genres
    genre_counter = Counter()

    # Paramètres pour le batching
    limit = 1000
    skip = 0

    while True:
        batch = client.list_movies(skip=skip, limit=limit, output_format="dict")
        if not batch:
            break

        # On extrait les genres du lot et les compte
        for movie in batch:
            genres = movie.get("genres", "")
            genre_list = genres.split("|") if genres else []
            genre_counter.update(genre_list)

        skip += limit
        time.sleep(0.5)  # Pour respecter l’API

    # Conversion du Counter en DataFrame
    genre_df = pd.DataFrame(genre_counter.items(), columns=["genre", "count"])
    genre_df = genre_df.sort_values("count", ascending=False).head(10)

    # Sauvegarde
    genre_df.to_parquet(genre_data_file, index=False, engine="fastparquet")
    with open(meta_file, "w") as f:
        json.dump({"movie_count": api_movie_count}, f)

# Affichage Plotly
fig = px.bar(
    genre_df,
    x="count",
    y="genre",
    title="Top 10 genres par nombre de films",
    labels={"genre": "Genre", "count": "Nombre de films"},
    color="count",
    color_continuous_scale="viridis",
    orientation='h'
)

fig.update_layout(
    yaxis={'categoryorder':'total ascending'},
    height=500
)

fig.show()

9829
Chargement des données depuis le cache...




```
# Ce texte est au format code
```

## Nombre total de films par année (basée sur le titre)

In [19]:
import re

# === Dossiers ===
#output_dir = Path("output")
#output_dir.mkdir(exist_ok=True)

yearly_data_file = output_dir / "movies_by_year.parquet"
meta_file = output_dir / "meta_movies_by_year.json"

# === Récupération du nombre total de films via analytics ===
#analytics = client.get_analytics()
api_movie_count = analytics.movie_count

# === Lecture du cache s’il existe ===
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cached_movie_count = meta.get("movie_count", 0)
else:
    cached_movie_count = 0

# === Utilisation du cache ou recalcul ===
if yearly_data_file.exists() and cached_movie_count == api_movie_count:
    print("Chargement des données depuis le cache...")
    df_yearly = pd.read_parquet(yearly_data_file)

else:
    print("Extraction des années depuis l’API...")

    # === Initialisation ===
    year_counter = Counter()
    skip = 0
    limit = 500
    year_pattern = re.compile(r"\((\d{4})\)$")

    while True:
        batch = client.list_movies(skip=skip, limit=limit, output_format="dict")
        if not batch:
            break

        for movie in batch:
            title = movie.get("title", "")
            match = year_pattern.search(title)
            if match:
                year = int(match.group(1))
                year_counter[year] += 1

        skip += limit
        time.sleep(0.5)

    # === Construction du DataFrame ===
    df_yearly = pd.DataFrame(sorted(year_counter.items()), columns=["year", "movie_count"])

    # === Sauvegarde du cache ===
    df_yearly.to_parquet(yearly_data_file, index=False, engine = "fastparquet")
    with open(meta_file, "w") as f:
        json.dump({"movie_count": api_movie_count}, f)

# === Affichage avec Plotly ===
fig = px.bar(
    df_yearly,
    x="year",
    y="movie_count",
    title="Nombre total de films par année (basé sur le titre)",
    labels={"year": "Année", "movie_count": "Nombre de films"},
)

fig.update_layout(
    xaxis_title="Année",
    yaxis_title="Nombre de films",
    height=500
)

fig.show()

Extraction des années depuis l’API...


In [20]:
df_yearly

,year,movie_count
0,1902,1
1,1903,1
2,1908,1
3,1915,1
4,1916,4
...,...,...
101,2014,277
102,2015,274
103,2016,218
104,2017,147


Top 20 des films par nombre d'évaluations

In [33]:
# === Dossiers ===
#output_dir = Path("output")
#output_dir.mkdir(exist_ok=True)

top_movies_file = output_dir / "top_movies_by_ratings.parquet"
meta_file = output_dir / "meta_top_movies.json"

# === Récupération des métriques API ===
#analytics = client.get_analytics()
api_movie_count = analytics.movie_count
api_rating_count = analytics.rating_count

# === Vérification du cache ===
if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
    cached_movie_count = meta.get("movie_count", 0)
    cached_rating_count = meta.get("rating_count", 0)
else:
    cached_movie_count = 0
    cached_rating_count = 0

# === Utilisation du cache ou recalcul ===
if (
    top_movies_file.exists()
    and cached_movie_count == api_movie_count
    and cached_rating_count == api_rating_count
):
    print("Chargement des données depuis le cache...")
    top_movies_df = pd.read_parquet(top_movies_file, engine="fastparquet")

else:
    print("Récupération des évaluations depuis l’API...")

    # === Initialisation des compteurs ===
    movie_rating_count = defaultdict(int)
    movie_rating_sum = defaultdict(float)

    # === Batching des ratings ===
    limit = 500
    skip = 0

    while True:
        batch = client.list_ratings(skip=skip, limit=limit, output_format="dict")
        if not batch:
            break

        for rating in batch:
            movie_id = rating["movieId"]
            score = rating["rating"]
            movie_rating_count[movie_id] += 1
            movie_rating_sum[movie_id] += score

        skip += limit
        time.sleep(0.5)

    # === Construction DataFrame des stats ===
    stats = [
        {
            "movieId": movie_id,
            "rating_count": movie_rating_count[movie_id],
            "avg_rating": movie_rating_sum[movie_id] / movie_rating_count[movie_id]
        }
        for movie_id in movie_rating_count
    ]

    stats_df = pd.DataFrame(stats)
    top_movies_df = stats_df.sort_values("rating_count", ascending=False).head(20)

    # === Ajout des titres de films via l’API ===
    movie_titles = {}
    for movie_id in top_movies_df["movieId"]:
        try:
            movie_data = client.get_movie(movie_id)
            movie_titles[movie_id] = movie_data.title
        except Exception as e:
            print(f"Erreur récupération titre movieId {movie_id} : {e}")
            movie_titles[movie_id] = f"Movie {movie_id}"

    top_movies_df["title"] = top_movies_df["movieId"].map(movie_titles)

    # === Sauvegarde dans le cache ===
    top_movies_df.to_parquet(top_movies_file, index=False)
    with open(meta_file, "w") as f:
        json.dump(
            {
                "movie_count": api_movie_count,
                "rating_count": api_rating_count
            },
            f
        )

# === Affichage avec Plotly ===
fig = px.bar(
    top_movies_df.sort_values("rating_count", ascending=True),  # Pour affichage de bas en haut
    x="rating_count",
    y="title",
    color="avg_rating",
    orientation="h",
    title="Top 20 des films par nombre d'évaluations",
    labels={
        "title": "Titre du film",
        "rating_count": "Nombre d'évaluations",
        "avg_rating": "Note moyenne"
    },
    color_continuous_scale="viridis"
)

fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    height=700
)

fig.show()

Chargement des données depuis le cache...


In [34]:
top_movies_df

,movieId,rating_count,avg_rating,title
0,356,329,4.164134,Forrest Gump (1994)
1,318,317,4.429022,"Shawshank Redemption, The (1994)"
2,296,307,4.197068,Pulp Fiction (1994)
3,593,279,4.161290,"Silence of the Lambs, The (1991)"
4,2571,278,4.192446,"Matrix, The (1999)"
5,260,251,4.231076,Star Wars: Episode IV - A New Hope (1977)
6,480,238,3.750000,Jurassic Park (1993)
7,110,237,4.031646,Braveheart (1995)
8,589,224,3.970982,Terminator 2: Judgment Day (1991)
9,527,220,4.225000,Schindler's List (1993)




```
# Ce texte est au format code
```

## Top tags utilisés par les utilisateurs de la plateforme

In [ ]:
#output_dir = Path("output")
#output_dir.mkdir(exist_ok=True)

tag_usage_file = output_dir / "user_tag_stats.parquet"
meta_file = output_dir / "meta_users_behavior.json"

# Récupération des métriques d’API pour surveiller les changements :
#analytics = client.get_analytics()
api_rating_count = analytics.rating_count
api_tag_count = analytics.tag_count

if meta_file.exists():
    with open(meta_file, "r") as f:
        meta = json.load(f)
else:
    meta = {}

cached_rating_count = meta.get("rating_count", 0)
cached_tag_count = meta.get("tag_count", 0)


# Tags souvent utilisés par certains utilisateurs

if tag_usage_file.exists() and cached_tag_count == api_tag_count:
    print("Chargement du cache : tags utilisés")
    tag_df = pd.read_parquet(tag_usage_file, engine =  "fastparquet")
else:
    print("Recalcul : tags utilisés")
    tag_counter = Counter()
    limit = 500
    skip = 0

    while True:
        batch = client.list_tags(skip=skip, limit=limit, output_format="dict")
        if not batch:
            break
        for tag in batch:
            tag_text = tag.get("tag", "")
            tag_counter[tag_text] += 1
        skip += limit
        time.sleep(0.5)

    tag_df = pd.DataFrame(tag_counter.items(), columns=["tag", "count"])
    tag_df = tag_df[tag_df["tag"].str.strip() != ""]
    tag_df = tag_df.sort_values("count", ascending=False).head(20)
    tag_df.to_parquet(tag_usage_file, index=False, engine = "fastparquet")

fig4 = px.bar(
    tag_df, x="count", y="tag", orientation="h",
    title="Top tags utilisés par les utilisateurs",
    labels={"count": "Nombre d’utilisations", "tag": "Tag"},
    color="count", color_continuous_scale="viridis"
)
fig4.update_layout(yaxis={'categoryorder': 'total ascending'})
fig4.show()

with open(meta_file, "w") as f:
    json.dump({
        "rating_count": api_rating_count,
        "tag_count": api_tag_count
    }, f)

Chargement du cache : tags utilisés


ArrowKeyError: A type extension with name pandas.period already defined